In [1]:
import torch
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

c:\Users\dlaptev\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
from modules.encoder import Encoder
enc = Encoder(1, tokenizer.vocab_size, 4, 2, dropout=0.1)

In [7]:
texts = [
    'Hello, ML&DS Community!',
    'Hello, Sanya!'
]

tokenized = tokenizer(texts, padding='max_length', max_length=8, truncation=True)
tokens = torch.tensor(tokenized['input_ids'])


In [8]:
enc(tokens)

tensor([[[-1.2117,  1.2371, -0.7202,  0.6947],
         [ 0.5334,  1.1733, -0.1891, -1.5176],
         [ 0.4326, -1.0453,  1.4316, -0.8189],
         [ 0.7642, -0.5555,  1.1377, -1.3464],
         [ 0.7931, -0.2159,  0.9679, -1.5452],
         [-1.0503,  1.1669,  0.8159, -0.9325],
         [-1.0983, -0.6712,  0.2612,  1.5083],
         [ 0.9103,  0.2229,  0.5468, -1.6801]],

        [[-1.2519,  1.5155,  0.0929, -0.3565],
         [ 0.4635,  1.2536, -0.2504, -1.4666],
         [ 0.0646, -1.4857,  1.3346,  0.0866],
         [ 0.4447, -1.2131,  1.3930, -0.6246],
         [ 0.4044,  1.1975, -0.0527, -1.5491],
         [-1.5701,  0.2962,  1.2004,  0.0734],
         [-0.2128, -1.2280, -0.1189,  1.5597],
         [-1.2336, -0.3636,  0.0670,  1.5301]]],
       grad_fn=<NativeLayerNormBackward0>)